In [1]:
import pandas as pd
import numpy as np
customers=pd.read_csv('recommend_1.csv')
transactions=pd.read_csv('trx_data.csv')

In [2]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [3]:
data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

In [4]:
data.head()

,customerId,productId,purchase_count
0,0,1.0,2
1,0,13.0,1
2,0,19.0,3
3,0,20.0,1
4,0,31.0,2


In [5]:
data['productId'] = data['productId'].astype(np.int64)

In [6]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [7]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix_norm = ((df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min()))
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [8]:
data_norm.head()

,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [9]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(data,test_size=0.2,random_state=0)
#train test size (106868, 3) (26717, 3)

In [10]:
import graphlab as gf
train_data = gf.SFrame(train)
test_data = gf.SFrame(test)

This non-commercial license of GraphLab Create for academic use is assigned to viveksood92@gmail.com and will expire on July 21, 2020.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\vivek\AppData\Local\Temp\graphlab_server_1563897271.log.0


In [11]:
train_data

customerId,productId,purchase_count
8342,2,1
11081,255,1
16786,50,1
24759,6,3
15142,1,1
7090,156,1
887,69,2
10694,20,1
529,89,1
23734,33,1


In [14]:
# We can define a function for this step as follows for split data

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = gf.SFrame(train)
    test_data = gf.SFrame(test)
    return train_data, test_data

In [15]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [16]:
train_data_dummy


customerId,productId,purchase_count,purchase_dummy
555,96,2,1
5116,253,1,1
7699,7,2,1
5649,224,1,1
10539,31,2,1
9183,46,1,1
20115,113,1,1
378,20,1,1
4114,86,1,1
1341,79,1,1


In [17]:
train_data_norm #check purpose

customerId,productId,scaled_purchase_freq
1500,19,0.0714285714286
8698,219,0.0
24222,26,0.571428571429
605,26,0.0
3147,240,0.0
9056,11,0.05
9213,0,0.0
20658,235,0.4
8579,84,0.0
4706,49,0.0


In [ ]:
'''We use 80:20 ratio for our train-test set size.
Our training portion will be used to develop a predictive model, while the other to evaluate the model's performance.
Now that we have three datasets with purchase counts, purchase dummy, and scaled purchase counts, we would like to split each.'''